In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re

%matplotlib inline

In [9]:
df4 = pd.read_excel('Linkedin_Extract_20230804.xlsx')
df = pd.read_excel('Linkedin_Extract_20240226.xlsx')

In [10]:
df = pd.concat([df, df4], ignore_index=True)
df.drop_duplicates(inplace = True)

In [11]:
df = df.dropna(subset=['name'])

In [12]:
df.drop_duplicates(subset='name', keep='first',inplace = True)

In [13]:
df.reset_index(drop = True,inplace =True)

In [14]:
df.shape

(7944, 679)

In [15]:
df['location'] = df['location'].apply(lambda x: str(x).split(',')[0].lower())

In [16]:
df['location'] = df['location'].str.replace('bengaluru', 'bangalore')

In [17]:
def replace_values(df, column_names):
    for col in column_names:
        df[col] = df[col].fillna('')
        df[col] = df[col].apply(lambda x: x.lower())
        df[col] = df[col].apply(lambda x: str(x).split(',')[0].lower())
        len1 = set(df[df[col].str.split().str.len() == 1][col].to_list())
        len2 = set(df[df[col].str.split().str.len() > 1][col].to_list())

        for i in len2:
            for j in len1:
                if pd.notna(i) and pd.notna(j) and j in i:
                    replacement_indices = df[df[col] == i].index
                    df.loc[replacement_indices, col] = j

In [18]:
columns_to_process = ['location']
replace_values(df, columns_to_process)

job_location_cols = [col for col in df.columns if 'job' in col and 'location' in col]
columns_to_process = job_location_cols
replace_values(df, columns_to_process)

In [19]:
df.shape

(7944, 679)

In [20]:
pattern = r'([a-zA-Z]{3} \d{4} - [a-zA-Z]{3} \d{4})'
mask = df[job_location_cols].apply(lambda col: col.str.contains(pattern, flags=re.IGNORECASE, regex=True)).any(axis=1)

C:\Users\PM\AppData\Local\Temp\ipykernel_16460\2340561830.py:2: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  mask = df[job_location_cols].apply(lambda col: col.str.contains(pattern, flags=re.IGNORECASE, regex=True)).any(axis=1)


In [21]:
df = df[~mask]
df.shape

(7886, 679)

# all location columns are done.

In [22]:
    # 'name', 'location',
    # 'title', 'org', 'job_duration', 'job_location',
    # 'institute', 'digree', 'digree_duration',
    # 'skill',
    # 'company_emp_count', 'company_industry',

In [23]:
df['name'] = df['name'].str.lower()

In [24]:
title_cols = [col for col in df.columns if 'title' in col]

In [25]:
def process_title(title):
    if isinstance(title, str):
        parts = title.split(',')[0]  # Split by comma and get the first part
        return(parts.lower())          # Convert to lowercase
    else:
        return(title)
df[title_cols] = df[title_cols].applymap(process_title)

In [26]:
unique_title = set()
for col in title_cols:
    unique_title.update(set(df[col].unique()))


# title cols...done

In [27]:
for column in df.columns:
    if df[column].dtype == 'object':  # Check if the column contains string values
        df[column] = df[column].str.lower().replace(r'[^\w\s]', '', regex=True).replace(r'fulltime|selfemployed|· self-employed|· full-time', '', regex=True).str.strip()

In [28]:
df.replace(np.nan,'NaN',inplace=True)

In [29]:
    # 'name', 'location',
    # 'title', 'org', 'job_duration', 'job_location',
    # 'institute', 'digree', 'digree_duration',
    # 'skill',
    # 'company_emp_count', 'company_industry',

In [30]:
org_cols = ([col for col in df.columns if 'org' in col])

In [31]:
df.shape

(7886, 679)

In [32]:
pattern = r'(\d{1,2}) yrs (\d{1,2}) mos'
# Create a boolean mask for matching rows
mask = df[org_cols].apply(lambda col: col.str.contains(pattern, flags=re.IGNORECASE, regex=True)).any(axis=1)
df = df[~mask]
df.shape

C:\Users\PM\AppData\Local\Temp\ipykernel_16460\3756124456.py:3: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  mask = df[org_cols].apply(lambda col: col.str.contains(pattern, flags=re.IGNORECASE, regex=True)).any(axis=1)


(7779, 679)

In [33]:
def process_title(title):
    if isinstance(title, str):
        parts = title.split('(')[0]  # Split by comma and get the first part
        return(parts.lower())          # Convert to lowercase
    else:
        return(title)
df[org_cols] = df[org_cols].applymap(process_title)

In [34]:
df[org_cols]

,org_0,org_0_sub_0,org_0_sub_1,org_0_sub_2,org_0_sub_3,org_0_sub_4,org_0_sub_5,org_0_sub_6,org_0_sub_7,org_0_sub_8,...,org_17_sub_0,org_17_sub_1,org_18,org_18_sub_0,org_18_sub_1,org_19,org_19_sub_0,org_19_sub_1,org_20,current_org_industry
0,swift infos,nan,nan,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
1,vidai labs,nan,nan,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
2,apnaklub,nan,nan,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
3,brewhouse tea brewing co,nan,nan,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
4,alteryx india,nan,nan,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7938,hira enterprise,nan,nan,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
7939,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
7940,auxo technology labs private limited,nan,nan,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
7941,ilyzly,nan,nan,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,retail apparel and fashion


# ORG done

In [35]:
# df.to_csv('df.csv',index = False)

In [36]:
# df=pd.read_csv('df.csv')

checkpoint

In [37]:
    # 'name', 'location',
    # 'title', 'org', 'job_duration', 'job_location',
    # 'institute', 'digree', 'digree_duration',
    # 'skill',
    # 'company_emp_count', 'company_industry',

In [38]:
# # Function to extract and convert duration to months
# def extract_duration_to_months(duration_str):
#     if pd.isna(duration_str):
#         return 0
#     pattern = r'(\d+)\s*(?:yr|y(?:r|ears))?.*?(\d+)\s*mo(?:s|nths)?|(\d+)\s*yr'

#     total_months = 0
#     matches = re.findall(pattern, duration_str)

#     for match in matches:
#         years, months, years_only = match
#         if years:
#             years = int(years)
#             total_months += years * 12
#         if months:
#             months = int(months)
#             total_months += months
#         if years_only:
#             years_only = int(years_only)
#             total_months += years_only * 12
#     print(years, months, years_only )

# Checkpoint

In [39]:
# df = pd.read_csv('df.csv')

In [40]:
job_duration_cols = [col for col in df.columns if 'job' in col and 'duration' in col]

In [41]:
df[job_duration_cols]

,job_0_duration,job_0_sub_0_duration,job_0_sub_1_duration,job_0_sub_2_duration,job_0_sub_3_duration,job_0_sub_4_duration,job_0_sub_5_duration,job_0_sub_6_duration,job_0_sub_7_duration,job_0_sub_8_duration,...,job_17_duration,job_17_sub_0_duration,job_17_sub_1_duration,job_18_duration,job_18_sub_0_duration,job_18_sub_1_duration,job_19_duration,job_19_sub_0_duration,job_19_sub_1_duration,job_20_duration
0,mar 2013 present 10 yrs 6 mos,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,sep 2022 present 1 yr,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,jun 2019 present 4 yrs 3 mos,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,may 2012 present 11 yrs 4 mos,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,aug 2019 present 4 yrs 1 mo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7938,apr 2017 present 6 yrs 5 mos,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7939,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7940,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7941,jun 2017 present 6 yrs 3 mos,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [42]:
def extract_duration_to_months(duration_str):
    if pd.isna(duration_str):
        return 'NAN'

    pattern1 = r'(\d+)\s*y(?:r|ears)?.*?(\d+)\s*mo(?:s|nths)?'
    pattern2 = r'(\d+)\s*(?:yr|y(?:r|ears))?.*?(\d+)\s*mo(?:s|nths)?|(\d+)\s*yr'

    total_months_1 = 0
    total_months_2 = 0

    matches1 = re.findall(pattern1, duration_str)
    for match in matches1:
        years, months = match
        if years:
            years = int(years)
            total_months_1 += years * 12
        if months:
            months = int(months)
            total_months_1 += months

    matches2 = re.findall(pattern2, duration_str)
    for match in matches2:
        years, months, years_only = match
        if years:
            years = int(years)
            total_months_2 += years * 12
        if months:
            months = int(months)
            total_months_2 += months
        if years_only:
            years_only = int(years_only)
            total_months_2 += years_only * 12
    l = max(total_months_1, total_months_2)

    tot = 0

    if not len(str(l)) > 4:
      tot = (l)
    else:
      tot = (min(total_months_1, total_months_2))

    return tot 

for cols in job_duration_cols:
  df[cols]=df[cols].apply(extract_duration_to_months)

In [43]:
df[job_duration_cols] = df[job_duration_cols].astype(float)

In [44]:
c = 0
for k,v in df[job_duration_cols].iterrows():
    if np.nansum(v.to_numpy())//12 > 50:
        c+=1
        print(c,k,df.loc[k,'name'])

1 17 devarajan chinnusamy
2 36 varadarajan narasimhan
3 58 vinay v
4 75 sudhir sethi
5 78 ashok kini hosdurg
6 86 appu sivamani
7 96 vinay d
8 98 arvind uppal
9 120 n r narayana murthy
10 173 manas m
11 196 hemant sharma
12 226 drramniwas prasher
13 244 manivannan us
14 334 dr manish chandra prabhakar
15 349 amitabh singh
16 374 sameer mehta
17 422 narendra b
18 468 davinder singh mendhiratta
19 490 shanker damodaran
20 598 bitra n rao
21 611 author rrachita gupta prof life mgmt coach thru astrology tarot psychology
22 672 jagadeesan ramamoorthy
23 753 younus mulla
24 894 yogesh kundra
25 933 prateek ahuja
26 939 dr l r sharma
27 972 arunprakash ganesan
28 1029 t n manoharan
29 1046 sumeet goel
30 1052 avinash ganu
31 1078 drjayesh pavra
32 1100 robin varghese
33 1174 punith kumar
34 1182 prashant pathak
35 1183 abraham lincoln
36 1191 tina sharma
37 1214 sunil shinde
38 1223 subramanian varadarajan
39 1230 roshan nazareth
40 1244 kamal k r
41 1256 vijayaraghavan gonuguntla
42 1267 com

In [45]:
for k,v in df[job_duration_cols].iterrows():
    df.at[k,'total_exp'] = np.nansum(v.to_numpy())//12

In [46]:
df['total_exp'] = df['total_exp'].astype(int)
df['total_exp'].replace(0, np.nan, inplace=True)
df['total_exp']

0       10.0
1        4.0
2        9.0
3       18.0
4        4.0
        ... 
7938    15.0
7939     NaN
7940     NaN
7941    12.0
7943    19.0
Name: total_exp, Length: 7779, dtype: float64

In [47]:
df[job_duration_cols]

,job_0_duration,job_0_sub_0_duration,job_0_sub_1_duration,job_0_sub_2_duration,job_0_sub_3_duration,job_0_sub_4_duration,job_0_sub_5_duration,job_0_sub_6_duration,job_0_sub_7_duration,job_0_sub_8_duration,...,job_17_duration,job_17_sub_0_duration,job_17_sub_1_duration,job_18_duration,job_18_sub_0_duration,job_18_sub_1_duration,job_19_duration,job_19_sub_0_duration,job_19_sub_1_duration,job_20_duration
0,126.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,51.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,136.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,49.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7938,77.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7939,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7940,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7941,75.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# duration done

In [48]:
institute_cols = [col for col in df.columns if 'institute' in col]

In [49]:
df[institute_cols]

,institute_0,institute_1,institute_2,institute_3,institute_4,institute_5,institute_6,institute_7,institute_8,institute_9,...,institute_11,institute_12,institute_13,institute_14,institute_15,institute_16,institute_17,institute_18,institute_19,institute_20
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,amrita vishwa vidyapeetham,adhiyamann college of engineering hosur,src memorial metric higher secondary school,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,harvard business school,indian institute of technology delhi,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,indian institute of management calcutta,delhi university,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,dy patil international school,poddar international college india,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7938,ceo,manager,business development manager,marketing manager,marketing assistant,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7939,all india institute of medical sciences,osmania university,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7940,ceo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7941,rome business school,hindustan university,don bosco matriculation higher secondary school,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [50]:
df.shape

(7779, 680)

In [51]:
def contains_unwanted_values(row):
    unwanted_values = ['ceo', 'sr engineer','founder','managing director']
    return any(val.lower() in str(row).lower() for val in unwanted_values)

# Create a boolean mask based on the condition
mask = df[institute_cols].apply(contains_unwanted_values, axis=1)
df = df[~mask]
df.shape

(7657, 680)

In [52]:
def process_title(title):
    if isinstance(title, str):
        parts = title.split(',')[0]  # Split by comma and get the first part
        return(parts.lower())          # Convert to lowercase
    else:
        return(title)
df[institute_cols] = df[institute_cols].applymap(process_title)

In [53]:
df[institute_cols].head()

,institute_0,institute_1,institute_2,institute_3,institute_4,institute_5,institute_6,institute_7,institute_8,institute_9,...,institute_11,institute_12,institute_13,institute_14,institute_15,institute_16,institute_17,institute_18,institute_19,institute_20
0,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
1,amrita vishwa vidyapeetham,adhiyamann college of engineering hosur,src memorial metric higher secondary school,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
2,harvard business school,indian institute of technology delhi,nan,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
3,indian institute of management calcutta,delhi university,nan,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
4,dy patil international school,poddar international college india,nan,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan


#instute cols done

In [54]:
    # 'name', 'location',
    # 'title', 'org', 'job_duration', 'job_location',
    # 'institute', 'digree', 'digree_duration',
    # 'skill',
    # 'company_emp_count', 'company_industry',

In [55]:
degree_cols = [col for col in df.columns if 'digree' in col and 'duration' not in col]

In [56]:
df.shape

(7657, 680)

In [57]:
df = df.astype('str')

In [58]:
pattern = r'(\d{4}) - (\d{4})'

# Create a boolean mask for matching rows
mask = df[degree_cols].apply(lambda col: col.str.contains(pattern, flags=re.IGNORECASE, regex=True)).any(axis=1)

# Filter out rows based on the mask
df = df[~mask]
df.shape

C:\Users\PM\AppData\Local\Temp\ipykernel_16460\1228204545.py:4: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  mask = df[degree_cols].apply(lambda col: col.str.contains(pattern, flags=re.IGNORECASE, regex=True)).any(axis=1)


(7657, 680)

In [59]:
pattern = r'([a-zA-Z]{3} \d{4}) - ([a-zA-Z]{3} \d{4})'
mask = df[degree_cols].apply(lambda col: col.str.contains(pattern, flags=re.IGNORECASE, regex=True)).any(axis=1)

# Filter out rows based on the mask
df = df[~mask]
df.shape

C:\Users\PM\AppData\Local\Temp\ipykernel_16460\1577341530.py:2: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  mask = df[degree_cols].apply(lambda col: col.str.contains(pattern, flags=re.IGNORECASE, regex=True)).any(axis=1)


(7657, 680)

In [60]:
def process_title(title):
    if isinstance(title, str):
        parts = title.split(',')[0]  # Split by comma and get the first part
        return(parts.lower())          # Convert to lowercase
    else:
        return(title)
df[degree_cols] = df[degree_cols].applymap(process_title)

def process_title(title):
    if isinstance(title, str):
        parts = title.split('(')[0]  # Split by comma and get the first part
        return(parts.lower())          # Convert to lowercase
    else:
        return(title)
df[degree_cols] = df[degree_cols].applymap(process_title)

In [61]:
df[degree_cols]

,digree_0,digree_1,digree_2,digree_3,digree_4,digree_5,digree_6,digree_7,digree_8,digree_9,...,digree_11,digree_12,digree_13,digree_14,digree_15,digree_16,digree_17,digree_18,digree_19,digree_20
0,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
1,mtech molecular medicine,btech biotechnology,hsc biology,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
2,master of business administration mba busines...,bachelor of technology btech,nan,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
3,mba post graduate diploma in management,bachelor of commerce honours degree,nan,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
4,master of business administration mba marketing,bachelor of commerce bcom,nan,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7932,dme,1991 1996,nan,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
7933,bachelors degree visual communication,nan,nan,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
7935,master of commerce accounts,diploma in java technologies android developer...,bachelor of commerce bcom accounting and finance,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
7939,doctor of medicine md biochemistry,mbbs medicine,nan,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan


# degree cols done

In [62]:
# 'digree_duration',
# 'skill',
# 'company_emp_count', 'company_industry',

In [63]:
degree_duration_cols = [col for col in df.columns if 'digree' in col and 'duration' in col]

In [64]:
df[degree_duration_cols]

,digree_0_duration,digree_1_duration,digree_2_duration,digree_3_duration,digree_4_duration,digree_5_duration,digree_6_duration,digree_7_duration,digree_8_duration,digree_9_duration,...,digree_11_duration,digree_12_duration,digree_13_duration,digree_14_duration,digree_15_duration,digree_16_duration,digree_17_duration,digree_18_duration,digree_19_duration,digree_20_duration
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020 2022,2013 2017,1998 2013,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2017 2019,2008 2012,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2004 2006,1998 2001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2014 2016,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7932,1996 1999,1991 1996,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7933,2006 2009,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7935,apr 2016 mar 2018,2011 2017,2013 2016,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7939,1986 1988,1978 1985,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [65]:
import re

def calculate_duration(duration_str):
    pattern = r'(\d{4}) - (\d{4})'
    matches = re.findall(pattern, duration_str)

    total_months = 0
    for start_year, end_year in matches:
      if start_year.isdigit() and end_year.isdigit():
        start_year = int(start_year)
        end_year = int(end_year)
        total_months += (end_year - start_year) * 12
      else:
        return "NAN"

    total_years = total_months / 12

    if total_years < 1:
        return "Less than 1 year"
    elif total_years <= 4:
        return f"{int(total_years)} years"
    else:
        return "Greater than 4 years"

df[degree_duration_cols] = df[degree_duration_cols].applymap(lambda x: calculate_duration(str(x)) if isinstance(x, str) else 'NAN')

# digree duration done

In [66]:
# 'skill',
# 'company_emp_count', 'company_industry',

In [67]:
comp_emp = [col for col in df.columns if 'company' in col and 'emp' in col]
# comp_emp.append('employee_count')

In [68]:
def process_column(x):
    if pd.notnull(x):
        x = re.sub(r'employees|employee|alumni|overview|alumnus| associated members|associated member', '', x)
        x = x.replace(',', '')
        if x:
            return float(x)
    return x

# Process columns
for column in comp_emp:
    df[column] = df[column].astype(str).apply(process_column)

In [69]:
df['company_2_emp_count'] = df['company_2_emp_count'].replace('', np.nan).astype(float)

In [70]:
df[comp_emp].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7657 entries, 0 to 7941
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   company_0_emp_count   4957 non-null   float64
 1   company_1_emp_count   4254 non-null   float64
 2   company_2_emp_count   3670 non-null   float64
 3   company_3_emp_count   3095 non-null   float64
 4   company_4_emp_count   2336 non-null   float64
 5   company_5_emp_count   1068 non-null   float64
 6   company_6_emp_count   714 non-null    float64
 7   company_7_emp_count   465 non-null    float64
 8   company_8_emp_count   264 non-null    float64
 9   company_9_emp_count   181 non-null    float64
 10  company_10_emp_count  104 non-null    float64
 11  company_11_emp_count  54 non-null     float64
 12  company_12_emp_count  33 non-null     float64
 13  company_13_emp_count  21 non-null     float64
 14  company_14_emp_count  17 non-null     float64
 15  company_15_emp_count 

# emp_count done

In [71]:
comp_ind = [col for col in df.columns if 'company' in col and 'industry' in col]

In [72]:
df[comp_ind]

,company_0_industry,company_1_industry,company_2_industry,company_3_industry,company_4_industry,company_5_industry,company_6_industry,company_7_industry,company_8_industry,company_9_industry,company_10_industry,company_11_industry,company_12_industry,company_13_industry,company_14_industry,company_15_industry,company_16_industry,company_17_industry,company_18_industry,company_19_industry
0,it services and it consulting,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,veterinary services,manufacturing,food and beverage services,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,wholesale,NaN,business consulting and services,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,food and beverage services,banking,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7932,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7933,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7935,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7939,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# cleaning is done

In [73]:
# df.drop_duplicates(inplace=True)
# df.reset_index(drop=True, inplace=True)
# df.to_excel('Linkedin_cleaned.xlsx',index = False)

In [74]:
# df = pd.read_excel('Linkedin_cleaned.xlsx')
# orgdf = df.copy()

In [75]:
# df = orgdf.copy()

In [76]:
sdf = pd.DataFrame()
titlecols = [col for col in df.columns if 'title' in col]
sdf['all_titles'] = df[titlecols].apply(lambda row: ','.join(row.dropna().astype(str)), axis=1)#--------------------------------------remove punctuvation
# sdf.drop(columns=titlecols, inplace=True)

# Title

In [77]:
titles = []
for i in sdf['all_titles']:
  for j in i.split(','):
    titles.append(j)
len(titles)

1018381

In [78]:
from collections import defaultdict
title_counts = defaultdict(int)

for title in titles:
    title_counts[title] += 1

# Step 3: Sort the names based on their counts
sorted_title = sorted(title_counts.keys(), key=lambda title: title_counts[title], reverse=True)
sorted_title

titlec = []
# Print the sorted names
for title in sorted_title:
    titlec.append(f"{title}: {title_counts[title]}")


In [79]:
titledict = {}

In [80]:
inlist = ['founder']
notinlist = ['co','ceo','chief executive officer','c.e.o']# founder
apl = set()
for i in sorted_title:
    if all(word in i for word in inlist) and all(word not in i for word in notinlist):
      apl.add(i)
titledict['founder'] = apl

inlist = ['founder','co']# co-founder
notinlist = ['ceo','chief executive officer','c.e.o','product manager']
apl = set()
for i in sorted_title:
    if all(word in i for word in inlist) and all(word not in i for word in notinlist):
      apl.add(i)
titledict['co-founder'] = apl

inlist = ['ceo','chief executive officer','c.e.o']# founder and ceo         -- after mapping founder, ceo if there is founder in the list yet. then they are founder and ceo
notinlist = ['founder','product manager']
apl = set()
for i in sorted_title:
    if any(word in i for word in inlist) and all(word not in i for word in notinlist):
      apl.add(i)
titledict['ceo'] = apl

inlist = ['cfo']# co-founder
notinlist = ['ceo','chief executive officer','c.e.o','chief operating officer','product manager','assistant']
apl = set()
for i in sorted_title:
    if all(word in i for word in inlist) and all(word not in i for word in notinlist):
      apl.add(i)
titledict['cfo'] = apl

inlist = ['product manager']# co-founder
notinlist = ['ceo','chief executive officer','c.e.o','chief operating officer','assistant']
apl = set()
for i in sorted_title:
    if all(word in i for word in inlist) and all(word not in i for word in notinlist):
      apl.add(i)
titledict['product manager'] = apl

inlist = ['project manager']# co-founder
notinlist = ['ceo','chief executive officer','c.e.o','chief operating officer','assistant']
apl = set()
for i in sorted_title:
    if all(word in i for word in inlist) and all(word not in i for word in notinlist):
      apl.add(i)
titledict['project manager'] = apl

inlist = ['head','hr']# co-founder
notinlist = ['ceo','chief executive officer','c.e.o','chief operating officer','assistant','branch','polychrome','andhra pradesh']
apl = set()
for i in sorted_title:
    if all(word in i for word in inlist) and all(word not in i for word in notinlist):
      apl.add(i)
titledict['head-hr'] = apl

# inlist = ['head','hr']# co-founder
# notinlist = ['ceo','chief executive officer','c.e.o','chief operating officer','assistant','branch','polychrome','andhra pradesh']
# apl = set()
# for i in sorted_title:
#     if all(word in i for word in inlist) and all(word not in i for word in notinlist):
#       apl.add(i)
# titledict['head-hr'] = apl
# apl

In [81]:
inlist = ['head','hr']# co-founder
notinlist = ['ceo','chief executive officer','c.e.o','chief operating officer','assistant','branch','polychrome','andhra pradesh']
apl = set()
for i in sorted_title:
    if all(word in i for word in inlist) and all(word not in i for word in notinlist):
      apl.add(i)
titledict['head-hr'] = apl
apl

{'advisor  hr head',
 'agm and hr head',
 'agmhr and head capital markets line of business',
 'area head  hr',
 'area head hr',
 'area headhr',
 'associate vice president  head hr  bpo',
 'associate vice president hr head od',
 'asst manager  hr as head  hr',
 'avp  head hr low voltage products india',
 'avp  hr and group head',
 'avp  human resources head of hr',
 'avp  human resources regional hr head',
 'avp and head  chrmc',
 'avp head of apac  hr business partners  talent acquisition',
 'avp hr head human capital  organisational transformation',
 'board director  hr head',
 'bu hr head  manager human resources',
 'business head uaebahriain',
 'business hr head',
 'career counselor  hr head',
 'chief manager  head hr',
 'chief manager for siemens and hr head for siemens enterprise communication pvt ltd',
 'chief managerhr head hrwest',
 'circle head hr',
 'circle hr head mumbai',
 'cofounder  head of hrhcm consulting',
 'consultanthead hr',
 'corporate head  hr',
 'corporate head h

In [82]:
def map_degree(value):
    for k, v in titledict.items():
        if value in v:
            return k
    return value
for col in titlecols:
  df[f'mapped_{col}'] = df[col].apply(map_degree)

C:\Users\PM\AppData\Local\Temp\ipykernel_16460\1005312149.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'mapped_{col}'] = df[col].apply(map_degree)
C:\Users\PM\AppData\Local\Temp\ipykernel_16460\1005312149.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'mapped_{col}'] = df[col].apply(map_degree)
C:\Users\PM\AppData\Local\Temp\ipykernel_16460\1005312149.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Conside

In [83]:
df.columns

Index(['name', 'location', 'org_0', 'title_0', 'job_0_duration',
       'job_0_location', 'org_0_sub_0', 'title_0_sub_0',
       'job_0_sub_0_duration', 'job_0_sub_0_location',
       ...
       'mapped_title_17', 'mapped_title_17_sub_0', 'mapped_title_17_sub_1',
       'mapped_title_18', 'mapped_title_18_sub_0', 'mapped_title_18_sub_1',
       'mapped_title_19', 'mapped_title_19_sub_0', 'mapped_title_19_sub_1',
       'mapped_title_20'],
      dtype='object', length=813)

In [84]:
sdf = pd.DataFrame()
maptitlecols = []
for i in range(len(titlecols)):
  maptitlecols.append(f'mapped_{titlecols[i]}')
sdf['all_titles'] = df[maptitlecols].apply(lambda row: ','.join(row.dropna().astype(str)), axis=1)#--------------------------------------remove punctuvation

In [85]:
titles = []
for i in sdf['all_titles']:
  for j in i.split(','):
    titles.append(j)
len(titles)

1018381

In [86]:
from collections import defaultdict
title_counts = defaultdict(int)

for title in titles:
    title_counts[title] += 1

# Step 3: Sort the names based on their counts
sorted_title = sorted(title_counts.keys(), key=lambda title: title_counts[title], reverse=True)
sorted_title

titlec = []
# Print the sorted names
for title in sorted_title:
    titlec.append(f"{title}: {title_counts[title]}")
titlec

['NaN: 983551',
 'founder: 1632',
 'ceo: 1496',
 'product manager: 1192',
 'head-hr: 909',
 'co-founder: 800',
 'project manager: 626',
 'director: 444',
 'cfo: 399',
 'managing director: 330',
 'founder  ceo: 326',
 'chief financial officer: 307',
 'intern: 272',
 'business analyst: 253',
 'chief technology officer: 237',
 'software engineer: 234',
 'chief marketing officer: 213',
 'manager: 207',
 'consultant: 181',
 'summer intern: 168',
 'hr manager: 158',
 'vice president: 152',
 'assistant manager: 142',
 'business development manager: 137',
 'account manager: 125',
 'general manager: 122',
 'founder and ceo: 120',
 'head of human resources: 115',
 'senior software engineer: 112',
 'senior manager: 111',
 'hr executive: 100',
 'management trainee: 93',
 'associate: 92',
 'partner: 89',
 'president: 87',
 'owner: 82',
 'business development executive: 80',
 'chief operating officer: 75',
 'cofounder  ceo: 71',
 'managing partner: 68',
 'human resources manager: 68',
 'team lead: 6

In [87]:
inlist = ['founder','c']
notinlist = []# founder might be founder and ceo.
apl = set()
for i in sorted_title:
    if all(word in i for word in inlist) and all(word not in i for word in notinlist):
      apl.add(i)
titledict['founder and ceo'] = apl

inlist = ['software','engineer']
notinlist = []
apl = set()
for i in sorted_title:
    if all(word in i for word in inlist) and all(word not in i for word in notinlist):
      apl.add(i)
titledict['software engineer'] = apl

inlist = ['intern']
notinlist = ['international','internal']
apl = set()
for i in sorted_title:
    if all(word in i for word in inlist) and all(word not in i for word in notinlist):
      apl.add(i)
titledict['internship'] = apl

inlist = ['business', 'analyst']
notinlist = ['intern']
apl = set()
for i in sorted_title:
    if all(word in i for word in inlist) and all(word not in i for word in notinlist):
      apl.add(i)
titledict['business analyst'] = apl

inlist = ['consultant']
notinlist = []
apl = set()
for i in sorted_title:
    if all(word in i for word in inlist) and all(word not in i for word in notinlist):
      apl.add(i)
titledict['consultant'] = apl

inlist = ['manager']
notinlist = ['assistant','account','asst','assit']
apl = set()
for i in sorted_title:
    if all(word in i for word in inlist) and all(word not in i for word in notinlist):
      apl.add(i)
titledict['manager'] = apl

inlist = ['manager','account']
notinlist = ['assistant','accounts','asst','assit']
apl = set()
for i in sorted_title:
    if all(word in i for word in inlist) and all(word not in i for word in notinlist):
      apl.add(i)
titledict['account manager'] = apl

inlist = ['vice', 'president']
notinlist = []
apl = set()
for i in sorted_title:
    if all(word in i for word in inlist) and all(word not in i for word in notinlist):
      apl.add(i)
titledict['vice president'] = apl

inlist = ['president',]
notinlist = ['vice', 'founder','ceo']
apl = set()
for i in sorted_title:
    if all(word in i for word in inlist) and all(word not in i for word in notinlist):
      apl.add(i)
titledict['president'] = apl

inlist = ['software', 'developer',]
notinlist = []
apl = set()
for i in sorted_title:
    if all(word in i for word in inlist) and all(word not in i for word in notinlist):
      apl.add(i)
titledict['software developer'] = apl
#------------------------------------------------------------------------------------------------------
inlist = ['cto']
notinlist = ['ecto','acto','ucto','octo','busi','inter']
apl = set()
for i in sorted_title:
    if all(word in i for word in inlist) and all(word not in i for word in notinlist):
      apl.add(i)
titledict['CTO'] = apl
#------------------------------------------------------------------------------------------------------
inlist = ['cmo']
notinlist = ['ecto','acto','ucto','octo','busi','inter']
apl = set()
for i in sorted_title:
    if all(word in i for word in inlist) and all(word not in i for word in notinlist):
      apl.add(i)
titledict['CMO'] = apl

In [88]:
titledict.keys()

dict_keys(['founder', 'co-founder', 'ceo', 'cfo', 'product manager', 'project manager', 'head-hr', 'founder and ceo', 'software engineer', 'internship', 'business analyst', 'consultant', 'manager', 'account manager', 'vice president', 'president', 'software developer', 'CTO', 'CMO'])

In [89]:
# titledict['founder and ceo'].update(titledict['founder'])
# del titledict['founder']
# titledict['founder and ceo'].update(titledict['ceo'])
# del titledict['ceo']
titledict['founder and ceo'].update(titledict['co-founder'])
del titledict['co-founder']
titledict['president'].update(titledict['vice president'])
del titledict['vice president']
titledict['software developer'].update(titledict['software engineer'])
del titledict['software engineer']
del titledict['manager']
del titledict['internship']

In [90]:
converted = pd.DataFrame.from_dict(titledict, orient='index').T
converted.to_excel('title_maps.xlsx',index = False)

In [91]:
unmap_title = set()
def map_degree(value):
    for k, v in titledict.items():
        if value in v:
            return k
    unmap_title.add(value)
    return np.nan
for col in titlecols:
  df[f'mapped_{col}'] = df[col].apply(map_degree)

In [92]:
unmap_title = {value for value in unmap_title if value == value}

In [93]:
pd.DataFrame(unmap_title,columns = ['titles']).to_excel('unmapped_titles.xlsx')

In [94]:
value_counts = {}

for col in df[titlecols].columns:
    counts = df[f'mapped_{col}'].value_counts()
    for value, count in counts.items():
        if value in value_counts:
            value_counts[value] += count
        else:
            value_counts[value] = count
value_counts

{'founder': 1632,
 'founder and ceo': 1528,
 'ceo': 1496,
 'head-hr': 910,
 'product manager': 1192,
 'cfo': 409,
 'project manager': 627,
 'president': 1019,
 'business analyst': 425,
 'consultant': 1101,
 'account manager': 318,
 'software developer': 635,
 'CMO': 20,
 'CTO': 46}

In [95]:
#T

In [96]:
converted = pd.DataFrame.from_dict(value_counts, orient='index',columns = ['counts'])
converted.index.name = 'Roles'
converted.to_excel('title.xlsx')
converted

,counts
Roles,
founder,1632
founder and ceo,1528
ceo,1496
head-hr,910
product manager,1192
cfo,409
project manager,627
president,1019
business analyst,425


# industry.

In [97]:
comp_ind = [col for col in df.columns if 'company' in col and 'industry' in col]
value_counts = {}

for col in df[comp_ind].columns:
    counts = df[col].value_counts()
    for value, count in counts.items():
        if value in value_counts:
            value_counts[value] += count
        else:
            value_counts[value] = count


In [98]:
sorted_vc = {k: v for k, v in sorted(value_counts.items(), key=lambda item: item[1], reverse=True)}

In [99]:
one_row_data = pd.DataFrame([sorted_vc]).T.reset_index()
one_row_data.columns = ['Value', 'Occurrences']

# Save the transposed DataFrame to an Excel file
# one_row_data.to_excel('data.xlsx', index=False)

In [100]:
industry_mapping = {
    "it services and it consulting": "Software",
    "financial services": "Financial Services",
    "software development": "Software",
    "technology, information and internet": "Software",
    "education administration programs": "Education",
    "advertising services": "Media",
    "business consulting and services": "Professional Services",
    "information technology & services": "Software",
    "hospitals and health care": "Healthcare",
    "higher education": "Education",
    "telecommunications": "Telecommunications",
    "manufacturing": "Manufacturing",
    "retail": "FMCG",
    "real estate": "Real Estate",
    "banking": "Financial Services",
    "motor vehicle manufacturing": "Manufacturing",
    "e-learning providers": "Education",
    "non-profit organizations": "Non-profit",
    "food and beverage services": "Hospitality",
    "construction": "Construction",
    "pharmaceutical manufacturing": "Manufacturing",
    "insurance": "Financial Services",
    "staffing and recruiting": "Professional Services",
    "retail apparel and fashion": "Consumer",
    "appliances, electrical, and electronics manufacturing": "Consumer",
    "human resources services": "Professional Services",
    "transportation, logistics, supply chain and storage": "Transport, supply chain and logistics",
    "hospitality": "Hospitality",
    "industrial machinery manufacturing": "Manufacturing",
    "education management": "Education",
    "wellness and fitness services": "Professional Services",
    "travel arrangements": "Hospitality",
    "media production": "Media",
    "entertainment providers": "Media",
    "oil and gas": "Resources",
    "architecture and planning": "Planning and Design",
    "education": "Education",
    "outsourcing and offshoring consulting": "Professional Services",
    "venture capital and private equity principals": "Financial Services",
    "mining": "Resources",
    "professional training and coaching": "Professional Services",
    "wholesale building materials": "Construction",
    "chemical manufacturing": "Manufacturing",
    "computers and electronics manufacturing": "Manufacturing",
    "accounting": "Professional Services",
    "consumer services": "Consumer",
    "events services": "Hospitality",
    "textile manufacturing": "Manufacturing",
    "government administration": "Government",
    "food and beverage manufacturing": "Manufacturing",
    "design services": "Planning and Design",
    "professional services": "Professional Services",
    "automation machinery manufacturing": "Manufacturing",
    "defense and space manufacturing": "Manufacturing",
    "human resources": "Professional Services",
    "broadcast media production and distribution": "Media",
    "non-profit organization management": "Non-profit",
    "newspaper publishing": "Publishing",
    "computer and network security": "Software",
    "investment management": "Financial Services",
    "research services": "Professional Services",
    "book and periodical publishing": "Publishing",
    "machinery manufacturing": "Manufacturing",
    "renewable energy semiconductor manufacturing": "Manufacturing",
    "online audio and video media": "Media",
    "apparel & fashion": "Consumer",
    "e-learning": "Education",
    "spectator sports": "Sports",
    "airlines and aviation": "Transport, supply chain and logistics",
    "biotechnology research": "Professional Services",
    "market research": "Media",
    "legal services": "Professional Services",
    "internet publishing": "Publishing",
    "public relations and communications services": "Professional Services",
    "retail luxury goods and jewelry": "Consumer",
    "leisure, travel & tourism": "Hospitality",
    "information services": "Software",
    "investment banking": "Financial Services",
    "semiconductor manufacturing": "Manufacturing",
}


In [101]:
sdf = pd.DataFrame()
sdf

""


In [102]:
def indus_val(value):
  for k,v in industry_mapping.items():
    if value == k:
      return v.lower()
  else:
    return 'others'.lower()

for col in comp_ind:
  df[f'mapped_{col}'] = df[col].apply(lambda x: indus_val(x) if pd.notna(x) else x)
df[comp_ind]

,company_0_industry,company_1_industry,company_2_industry,company_3_industry,company_4_industry,company_5_industry,company_6_industry,company_7_industry,company_8_industry,company_9_industry,company_10_industry,company_11_industry,company_12_industry,company_13_industry,company_14_industry,company_15_industry,company_16_industry,company_17_industry,company_18_industry,company_19_industry
0,it services and it consulting,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,veterinary services,manufacturing,food and beverage services,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,wholesale,NaN,business consulting and services,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,food and beverage services,banking,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7932,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7933,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7935,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7939,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [103]:
value_counts = {}

for col in df[comp_ind].columns:
    counts = df[f'mapped_{col}'].value_counts()
    for value, count in counts.items():
        if value in value_counts:
            value_counts[value] += count
        else:
            value_counts[value] = count
value_counts

{'others': 136670,
 'software': 5213,
 'financial services': 2307,
 'professional services': 2045,
 'manufacturing': 1789,
 'education': 1120,
 'media': 856,
 'hospitality': 477,
 'fmcg': 352,
 'consumer': 284,
 'real estate': 312,
 'healthcare': 379,
 'construction': 263,
 'telecommunications': 423,
 'resources': 234,
 'planning and design': 116,
 'publishing': 165,
 'transport, supply chain and logistics': 60,
 'sports': 33,
 'government': 42}

In [104]:
converted = pd.DataFrame.from_dict(value_counts, orient='index').T
converted.to_csv('industry.csv',index = False)

In [105]:
df.columns

Index(['name', 'location', 'org_0', 'title_0', 'job_0_duration',
       'job_0_location', 'org_0_sub_0', 'title_0_sub_0',
       'job_0_sub_0_duration', 'job_0_sub_0_location',
       ...
       'mapped_company_10_industry', 'mapped_company_11_industry',
       'mapped_company_12_industry', 'mapped_company_13_industry',
       'mapped_company_14_industry', 'mapped_company_15_industry',
       'mapped_company_16_industry', 'mapped_company_17_industry',
       'mapped_company_18_industry', 'mapped_company_19_industry'],
      dtype='object', length=833)

# size of the company

In [106]:
comp_emp_count = [col for col in df.columns if 'company' in col and 'emp' in col]
df[comp_emp_count]

,company_0_emp_count,company_1_emp_count,company_2_emp_count,company_3_emp_count,company_4_emp_count,company_5_emp_count,company_6_emp_count,company_7_emp_count,company_8_emp_count,company_9_emp_count,company_10_emp_count,company_11_emp_count,company_12_emp_count,company_13_emp_count,company_14_emp_count,company_15_emp_count,company_16_emp_count,company_17_emp_count,company_18_emp_count,company_19_emp_count
0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2.0,3013.0,237.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,198.0,NaN,21451.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,21.0,178351.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7932,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7933,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7935,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7939,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [107]:
def categorize_employee_count(emp_count): # 1-3.3-9,250-1000, 1000-5000,5000+
    if np.isnan(emp_count):
        return "Unknown"
    elif (emp_count == 1) or (emp_count < 3):
        return "1-3"
    elif emp_count <= 9:
        return "3-9"
    elif emp_count <= 50:
        return "9-50"
    elif emp_count <= 250:
        return "50-250"
    elif emp_count <= 1000:
        return "250-1000"
    elif emp_count <= 5000:
        return "1000-5000"
    else:
        return "5000+"

for col in df[comp_emp_count].columns:
  df[f'mapped_{col}'] = df[col].apply(categorize_employee_count)

In [108]:
df[comp_emp_count].shape

(7657, 20)

In [109]:
value_counts = {}

for col in df[comp_emp_count].columns:
    counts = df[f'mapped_{col}'].value_counts()
    for value, count in counts.items():
        if value in value_counts:
            value_counts[value] += count
        else:
            value_counts[value] = count
value_counts

{'Unknown': 131881,
 '5000+': 7082,
 '9-50': 2547,
 '50-250': 3070,
 '3-9': 1497,
 '1000-5000': 3144,
 '250-1000': 2798,
 '1-3': 1121}

In [110]:
converted = pd.DataFrame.from_dict(value_counts, orient='index').T
converted.to_excel('size_of_the_company.xlsx',index = False)

# institurte cols

In [111]:
df

,name,location,org_0,title_0,job_0_duration,job_0_location,org_0_sub_0,title_0_sub_0,job_0_sub_0_duration,job_0_sub_0_location,...,mapped_company_10_emp_count,mapped_company_11_emp_count,mapped_company_12_emp_count,mapped_company_13_emp_count,mapped_company_14_emp_count,mapped_company_15_emp_count,mapped_company_16_emp_count,mapped_company_17_emp_count,mapped_company_18_emp_count,mapped_company_19_emp_count
0,prasad chandran,palakkadu,swift infos,founder,126.0,thrissur,nan,NaN,0.0,,...,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown
1,tamilmani selvan,erode,vidai labs,founder,12.0,coimbatore,nan,NaN,0.0,,...,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown
2,shruti shruti,bangalore,apnaklub,cofounder and ceo,51.0,bengaluru,nan,NaN,0.0,,...,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown
3,siddharth jain,delhi,brewhouse tea brewing co,founder,136.0,delhi,nan,NaN,0.0,,...,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown
4,swapnil more,bangalore,alteryx india,founder,49.0,bengaluru,nan,NaN,0.0,,...,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7932,santhosh l,india,pongalur dairy,managing partner,74.0,,nan,NaN,0.0,,...,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown
7933,aarokia james raja ajr,chennai,aarokia james has verified their government id,government id,0.0,,nan,NaN,0.0,,...,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown
7935,rohit telwade,thane,warlox studios,founder ceo,29.0,vasai virar,nan,NaN,0.0,,...,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown
7939,sunil tadepalli,delhi,nan,NaN,0.0,,nan,NaN,0.0,,...,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown


In [112]:
college_tiers_df = pd.read_excel('Mapping of tiers of colleges.xlsx')
college_tiers_df.drop(columns = ['Institute ID','Score'],inplace = True)

In [113]:
pattern = r'^(.*?) - (.*)$'

left_values = []
right_values = []

# Iterate through the 'Name' column and conditionally extract values
for name in college_tiers_df['Name']:
    if ('[' in name) and ('-' in name):
        left, right = name.split('[')
        left_values.append(left)
        right_values.append(right)
    else:
        left_values.append(None)
        right_values.append(None)

# Add the extracted values as new columns
college_tiers_df['proper_names'] = left_values
college_tiers_df['subname and loc'] = right_values

In [114]:
college_tiers_df['proper_names'] = college_tiers_df['proper_names'].str.replace('-','').str.strip()
college_tiers_df['proper_names'] = college_tiers_df['proper_names'].str.replace(',','')

In [115]:
namelis = []
college_tiers_df
for name in college_tiers_df['Name']:
    if not ('[' in name):
      namelis.append(name)
    else:
      namelis.append(None)
for i in range(len(college_tiers_df['proper_names'])):
  if college_tiers_df['proper_names'][i] == None:
    college_tiers_df['proper_names'][i] = namelis[i]

C:\Users\PM\AppData\Local\Temp\ipykernel_16460\4186195815.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  college_tiers_df['proper_names'][i] = namelis[i]


In [116]:
college_tiers_df['subname'] = college_tiers_df['subname and loc'].str.split(']')
college_tiers_df['subname'] = college_tiers_df['subname'].apply(lambda x: x[0] if isinstance(x, list) else None)

In [117]:
college_tiers_df['proper_names'] = college_tiers_df['proper_names'].str.replace('more details \|','')

C:\Users\PM\AppData\Local\Temp\ipykernel_16460\1888528338.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  college_tiers_df['proper_names'] = college_tiers_df['proper_names'].str.replace('more details \|','')


In [118]:
college_tiers_df['proper_names'] = college_tiers_df['proper_names'].apply(lambda x: (re.sub(r'\biit\b', 'indian institute of technology', x) if pd.notna(x) else x))
college_tiers_df['proper_names'] = college_tiers_df['proper_names'].apply(lambda x: (re.sub(r'\bnit\b', 'national institute of technology', x) if pd.notna(x) else x))
college_tiers_df['proper_names'] = college_tiers_df['proper_names'].apply(lambda x: (re.sub(r'\biiit\b', 'indian institute of information technology', x) if pd.notna(x) else x))

In [119]:
institute_cols = [col for col in df.columns if 'institute' in col]
for col in institute_cols:
    df[col] = df[col].apply(lambda x: x.split('-')[0] if '-' in str(x) else x)

In [120]:
for col in df[institute_cols].columns:
  df[col] = df[col].apply(lambda x: (re.sub(r'\biit\b', 'indian institute of technology', x) if pd.notna(x) else x))
  df[col] = df[col].apply(lambda x: (re.sub(r'\bnit\b', 'national institute of technology', x) if pd.notna(x) else x))
  df[col] = df[col].apply(lambda x: (re.sub(r'\biiit\b', 'indian institute of information technology', x) if pd.notna(x) else x))
  df[col] = df[col].apply(lambda x: (x.replace(',','')) if pd.notna(x) else x)
  df[col] = df[col].apply(lambda x: (x.replace('-','')) if pd.notna(x) else x)

In [121]:
college_tiers_df['proper_names'] = college_tiers_df['proper_names'].str.replace(',','')
college_tiers_df['proper_names'] = college_tiers_df['proper_names'].str.lower()

In [122]:
df = df.reset_index(drop=True)

In [123]:
sdf = df[institute_cols].copy()
sdf

,institute_0,institute_1,institute_2,institute_3,institute_4,institute_5,institute_6,institute_7,institute_8,institute_9,...,institute_11,institute_12,institute_13,institute_14,institute_15,institute_16,institute_17,institute_18,institute_19,institute_20
0,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
1,amrita vishwa vidyapeetham,adhiyamann college of engineering hosur,src memorial metric higher secondary school,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
2,harvard business school,indian institute of technology delhi,nan,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
3,indian institute of management calcutta,delhi university,nan,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
4,dy patil international school,poddar international college india,nan,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7652,psg college of technology,mani higher secondary school,nan,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
7653,loyola college,nan,nan,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
7654,university of mumbai,niit,st ggcollege,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
7655,all india institute of medical sciences,osmania university,nan,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan


In [124]:
def corr_ins_names(col, inde): # college names mapping to proper names based on subset.
    ind_sub = {}
    value = df[col][inde]
    prl = college_tiers_df['proper_names'].unique().tolist()

    if isinstance(value, str):
        value_set = set(value.split(' '))
        
        for name in range(len(prl)):
            name_set = set(prl[name].split(' '))
            
            if name_set.issubset(value_set) or value_set.issubset(name_set):
                sub_mat_lev = len(name_set.intersection(value_set))
                maxlen = max(len(name_set), len(value_set))

                diff = abs(sub_mat_lev - maxlen)
                if not diff > 2:
                    ind_sub[name] = sub_mat_lev

        ind_sub = dict(sorted(ind_sub.items(), key=lambda item: item[1]))

        if ind_sub:
            key = list(ind_sub.keys())[0]
            sdf.at[inde, col] = prl[key]
        else:
            sdf.at[inde, col] = 'others'


institute_cols = sdf.columns.intersection(df.columns)
for col in institute_cols:
    for i in range(len(df[col])):
        corr_ins_names(col, i)


In [125]:
# # send the df index...
# # k or v (ind of proper name, sub match level)
# # lessthan 2 set to None

# def corr_ins_names(col,inde):
#     ind_sub = {}
#     value = df[col][inde]
#     prl = college_tiers_df['proper_names'].tolist()

#     if isinstance(value, str):
#       for name in range(len(prl)):
#         if (set(prl[name].split(' ')).issubset(set(value.split(' ')))) or (set(value.split(' ')).issubset(set(prl[name].split(' ')))):

#           sub_mat_lev = len((set(prl[name].split(' '))).intersection(set(value.split(' '))))
#           maxlen = max( len(set(prl[name].split(' '))), len(set(value.split(' '))) )

#           diff = abs(sub_mat_lev - maxlen)
#           if not diff > 2:
#             ind_sub.update({name:sub_mat_lev})
#       ind_sub = dict(sorted(ind_sub.items(), key=lambda item: item[1]))
#       if ind_sub != {}:
#         key = list(ind_sub.keys())[0]
#         sdf[col][inde] = prl[key]
#       else:
#         sdf[col][inde] = 'others'

# for col in sdf[institute_cols].columns:
#   for i in range(len(df[institute_cols][col])):
#     corr_ins_names(col,i)

In [126]:
odf = df[institute_cols].copy()

In [127]:
for col in sdf.columns:
  odf[f'mapped_{col}'] = sdf[col]
for col in sdf.columns:
  odf[col] = sdf[col] # orginal cols here are for tier mappings.

In [128]:
odf

,institute_0,institute_1,institute_2,institute_3,institute_4,institute_5,institute_6,institute_7,institute_8,institute_9,...,mapped_institute_11,mapped_institute_12,mapped_institute_13,mapped_institute_14,mapped_institute_15,mapped_institute_16,mapped_institute_17,mapped_institute_18,mapped_institute_19,mapped_institute_20
0,others,others,others,others,others,others,others,others,others,others,...,others,others,others,others,others,others,others,others,others,others
1,amrita vishwa vidyapeetham,others,others,others,others,others,others,others,others,others,...,others,others,others,others,others,others,others,others,others,others
2,others,indian institute of technology delhi,others,others,others,others,others,others,others,others,...,others,others,others,others,others,others,others,others,others,others
3,indian institute of management,university of delhi,others,others,others,others,others,others,others,others,...,others,others,others,others,others,others,others,others,others,others
4,others,others,others,others,others,others,others,others,others,others,...,others,others,others,others,others,others,others,others,others,others
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7652,others,others,others,others,others,others,others,others,others,others,...,others,others,others,others,others,others,others,others,others,others
7653,loyola college,others,others,others,others,others,others,others,others,others,...,others,others,others,others,others,others,others,others,others,others
7654,mumbai university,others,others,others,others,others,others,others,others,others,...,others,others,others,others,others,others,others,others,others,others
7655,institute of medical sciences,osmania university,others,others,others,others,others,others,others,others,...,others,others,others,others,others,others,others,others,others,others


In [129]:
def tier_map(col,inde):
  value = odf[col][inde]
  prl = college_tiers_df['proper_names'].tolist()
  if isinstance(value, str):
      for name in range(len(prl)):
        if value == prl[name]:
          odf[col][inde] = college_tiers_df['Tier'][name]
          return
      else:
        odf[col][inde] = 4

for col in odf[institute_cols].columns:
  for i in range(len(odf[institute_cols][col])):
    tier_map(col,i)

In [130]:
odf.rename(columns = {col:f'tier_mapping_{col}' for col in odf[institute_cols].columns},inplace= True)

In [131]:
# for col in df[institute_cols].columns:
#   odf[f'tiers_{col}'] = df[col]

In [132]:
# for i in df[institute_cols]:
#   print(df[i].unique())


In [133]:
for col in odf.columns:
  df[col] = odf[col]

In [134]:
df.columns

Index(['name', 'location', 'org_0', 'title_0', 'job_0_duration',
       'job_0_location', 'org_0_sub_0', 'title_0_sub_0',
       'job_0_sub_0_duration', 'job_0_sub_0_location',
       ...
       'mapped_institute_11', 'mapped_institute_12', 'mapped_institute_13',
       'mapped_institute_14', 'mapped_institute_15', 'mapped_institute_16',
       'mapped_institute_17', 'mapped_institute_18', 'mapped_institute_19',
       'mapped_institute_20'],
      dtype='object', length=895)

In [135]:
rename_mapping = {1: 'tier-1', 2: 'tier-2', 3: 'tier-3', 4: 'tier-4'}
df[institute_cols] = df[institute_cols].replace(rename_mapping)

In [136]:
value_counts = {}

for col in df[institute_cols].columns:
    counts = df[f'tier_mapping_{col}'].value_counts()
    for value, count in counts.items():
        if value in value_counts:
            value_counts[value] += count
        else:
            value_counts[value] = count
value_counts

#{'tier-4': 3251, 'tier-1': 792, 'tier-3': 472, 'tier-2': 480}

{4: 155994, 1: 1808, 3: 1474, 2: 1521}

In [137]:
converted = pd.DataFrame.from_dict(value_counts, orient='index').T
rename_mapping = {1: 'tier-1', 2: 'tier-2', 3: 'tier-3', 4: 'tier-4'}
converted.rename(columns = rename_mapping,inplace = True)
converted.to_excel('institute.xlsx',index = False)

In [138]:
converted

,tier-4,tier-1,tier-3,tier-2
0,155994,1808,1474,1521


# Degree cols correction

In [139]:
degree_cols = [col for col in df.columns if 'digree' in col and 'duration' not in col]
# sdf['all_degree'] = df[degree_cols].apply(lambda row: ','.join(row.dropna().astype(str)), axis=1)

# degrees = []
# for i in sdf['all_degree']:
#   for j in i.split(','):
#     degrees.append(j)
# print(len(degrees))

# degree_counts = defaultdict(int)

# for degree in degrees:
#     degree_counts[degree] += 1

# # Step 3: Sort the names based on their counts
# sorted_degrees = sorted(degree_counts.keys(), key=lambda degree: degree_counts[degree], reverse=True)

In [140]:
degrees = set()
for ind,val in df[degree_cols].iterrows():
    val = val.to_list()
    degrees.update(set(val))
len(degrees)

7807

In [141]:
degree_dict = {
    'MBA': {'m.b.a.', 'master of business administration - mba', 'pgdbm', 'pgdm -marketing', 'mba.', 'mba finance', 'pgdm-abpm', 'pgdm', 'post graduate diploma in business management', 'master of business administration', 'post graduate program in management', 'mba - master of business administration', 'm.b.a', 'mba for executive', 'master in business administration', 'pgpm', 'post graduate programme in management', 'gold medalist m.b.a', 'mba'},
    'BE': {'bachelor of technology', 'be', 'bachelor of engineering - be', 'b. tech', 'b tech', 'b.e', 'bachelor of engineering', "bachelor's in engineering", 'b.tech', "bachelor's of technology", 'bachelor of technology - b. tech', 'b- tech', 'btech', 'bachelor of technology - btech', 'be - bachelor of engineering', 'b.e.', 'b.tech.', 'b. tech.'},
    # 'Bachelors Degree': {'bachelors', 'undergraduate', 'bachelor’s degree', "bachelor's degree"},
    'BCOM': {'bachelor of commerce ', 'b com', 'bcom', 'bachelor of commerce with honours - b.com', 'commerce', 'b.com.', 'b.com', 'bachelor of commerce', 'bcom - bachelor of commerce', "b'com", 'bachelor of commerce - bcom', 'b. com ', 'b. com', 'business/commerce'},
    'BBA': {'bachelor of business administration - bba', 'bachelor of business administration', 'bba', 'bba - finance and international business'},
    'CA': {'accounting and finance', 'chartered accountancy', 'chartered accountant', 'ca', 'chartered accountants', 'c.a.'},
    # "Masters Degree": {'master’s degree', "master's degree"},
    'PGDBA': {'pgdba'},
    'MTECH': {'master of technology', 'mtech', 'm.tech.', 'm.tech', 'master of technology - mtech'},
    'MCOM': {'master of commerce', 'm.com', 'b com post graduate'},
    'BA': {'business administration and management', 'bachelor of arts', 'ba', 'bachelor of arts - ba', 'ba programming', 'b.a.'},
    'MA': {'master of arts', 'm.a.', 'm.a. economics', 'master of arts - ma'},
    'MCA': {'master of computer applications - mca', 'master of computer applications', 'm.c.a', 'mca'},
    'BCA': {'bachelor of computer applications', 'bachelor of computer application', 'bachelor of computer aplication', 'bca', 'bachelor of computer application- b.c.a', "bachelor's of computer application", 'computer science'},
    'BSC': {'bsc', 'bachelor of science', 'b. sc.', 'bsc.it', 'b.sc.', 'bachelor of science - bs', 'b.sc', 'bachelor of science - b. sc.'},
    'MSC': {'msc in energy and resource management', 'master of science', 'msc - it', 'm.sc.', 'm.sc'},
    'MS': {'ms', 'master of science - ms', 'msc', 'msc in international construction management', 'm.s.'},
    'Bachelor of Laws': {'bachelor of laws - llb', 'bachelor of laws'},
    'Master of Laws': {'master of laws - llm'}
}

In [142]:
anyinlist = ['busi']
allinlist = ['mast','admi']
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in degrees:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
degree_dict['MBA'].update(apl)
#------------------------------------------------------------------------------------------------------
anyinlist = ['mba']
allinlist = []
notinlist = ['imba','umb','mbae']
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in degrees:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
degree_dict['MBA'].update(apl)
#------------------------------------------------------------------------------------------------------
anyinlist = ['be']
allinlist = []
notinlist = ['imba','umb','mbae','bed','mber','bel','bec','beh','ber','bet','bep','mbe','bey']
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in degrees:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
degree_dict['BE'].update(apl)
#------------------------------------------------------------------------------------------------------
anyinlist = ['bach']
allinlist = ['engi']
notinlist = ['civil','chemi','compu','mechan']
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in degrees:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
degree_dict['BE'].update(apl)
#------------------------------------------------------------------------------------------------------
# anyinlist = ['bach']
# allinlist = ['degre']
# notinlist = ['econo','engin','phar','scien','b.ed','mech']
# # notinlist.extend( list(skills_edited['C Programming Language']))
# # notinlist.remove('c')

# apl = set()
# for skill in degrees:
#     if isinstance(skill,str):    
#         if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
#             apl.add(skill)
# apl
# degree_dict['Bachelors Degree'].update(apl)
#------------------------------------------------------------------------------------------------------
anyinlist = ['b']
allinlist = ['com']
notinlist = ['econo','engin','phar','scien','b.ed','mech','comp','mcom','m com','ybco','commu','m.com']
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in degrees:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
degree_dict['BCOM'].update(apl)
#------------------------------------------------------------------------------------------------------
anyinlist = ['bba']
allinlist = []
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in degrees:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
degree_dict['BBA'].update(apl)
#------------------------------------------------------------------------------------------------------
anyinlist = ['ca ','char']
allinlist = []
notinlist = ['mca','bca','cs','dca','analy']
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in degrees:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
degree_dict['CA'].update(apl)
#------------------------------------------------------------------------------------------------------
# anyinlist = ['maste',]
# allinlist = ['deg']
# notinlist = ['mca','bca','cs','dca','analy','comp']
# # notinlist.extend( list(skills_edited['C Programming Language']))
# # notinlist.remove('c')

# apl = set()
# for skill in degrees:
#     if isinstance(skill,str):    
#         if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
#             apl.add(skill)
# apl
# degree_dict["Masters Degree"].update(apl)
#------------------------------------------------------------------------------------------------------
anyinlist = ['pgdba',]
allinlist = []
notinlist = ['mca','bca','cs','dca','analy','comp']
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in degrees:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
degree_dict['PGDBA'].update(apl)
#------------------------------------------------------------------------------------------------------
anyinlist = ['m',]
allinlist = ['tech']
notinlist = ['mca','bca','cs','dca','analy','comp','ma','am','otec','dm','em','sme','ms','mba','men','mic']
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in degrees:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
degree_dict['MTECH'].update(apl)
#------------------------------------------------------------------------------------------------------
anyinlist = ['m','master']
allinlist = ['com']
notinlist = ['b','bat','comp','grad','math','cial','xii','comn','comu','commu','ecom','12th','high','isc','commi','shri','www.']
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in degrees:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
degree_dict['MCOM'].update(apl)
#------------------------------------------------------------------------------------------------------
anyinlist = ['ba','art']
allinlist = ['ba']
notinlist = ['eba','com','ban','bal','bat','sci','bba','be','bar','bas','bay','mba','bam','pba','dba','bai','bab','sba','y ba','engi','tech',
             'mark','desi','busi','phar','agri','law','journal','dent','arch','deg','app','hosp','anim','edu','eco','medi','hote','lab','manag','corp',
             'soci','plan','acco','ocu','auto','bacca','tour','mast']
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in degrees:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
degree_dict['BA'].update(apl)
#------------------------------------------------------------------------------------------------------
anyinlist = ['ma','art']
allinlist = ['ma']
notinlist = ['eba','com','ban','bal','bat','sci','bba','be','bar','bas','bay','mba','bam','pba','dba','bai','bab','sba','y ba','engi','tech',
             'mark','desi','busi','phar','agri','law','journal','dent','arch','deg','app','hosp','anim','edu','eco','medi','hote','lab','manag','corp',
             'soci','plan','acco','ocu','auto','bacca','tour','oma','cma','man','ama','mad','mat','ima','mac','coac','mak','mar','pma','max','dma','pma',
             'may','nlp','phil','rma','gma','neg','mah','sal','dir','inter','slo','apr','fi','fa','heal','immu','publi','hr','it','tequ','execut','spec']
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in degrees:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
degree_dict['MA'].update(apl)
#------------------------------------------------------------------------------------------------------
anyinlist = ['mca','compu']
allinlist = []
notinlist = ['bach','bsc','b.sc','b sc','info','engi','12','advan','tech','scien','cloud','netwo','math','human',]
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in degrees:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
degree_dict['MCA'].update(apl)
#------------------------------------------------------------------------------------------------------
anyinlist = ['ca','compu']
allinlist = ['b']
notinlist = ['info','engi','12','advan','tech','scien','cloud','netwo','math','human','edu','mass','memb','deci','bcom','b.com',
             'b sc','b.sc','commu','inqu','interi','oca','car','cal','cai','cam','cate','cert','can','sca','cap']
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in degrees:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
degree_dict['BCA'].update(apl)
#------------------------------------------------------------------------------------------------------
anyinlist = ['scie','sc']
allinlist = ['b']
notinlist = ['bcom','b.com','msc','m sc','m.sc',
             'commu','inqu','interi','oca','car','cal','cai','cam','cate','cert','can','sca','cap','x','sch','basc','b.r',
             'data','19','ssc','subj','cious','logi']
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in degrees:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
degree_dict['BSC'].update(apl)
#------------------------------------------------------------------------------------------------------
anyinlist = ['scie','sc']
allinlist = ['m']
notinlist = ['bcom','b.com','msc','bsc','b.sc','b sc','tech','12','bach','phd','diplo',
             'commu','inqu','interi','oca','car','cal','cai','cam','cate','cert','can','sca','cap','x','sch','basc','b.r',
             'data','19','ssc','subj','cious','logi','mme']
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in degrees:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
degree_dict['MSC'].update(apl)
#------------------------------------------------------------------------------------------------------
anyinlist = ['scie','sc','ms']
allinlist = ['ms']
notinlist = ['bcom','b.com','msc','bsc','b.sc','b sc','tech','12','bach','phd','diplo',
             'commu','inqu','interi','oca','car','cal','cai','cam','cate','cert','can','sca','cap','x','sch','basc','b.r',
             'data','19','ssc','subj','cious','logi','mme','clai','ems','ams','hms','msw','mms','bms','msb','mse','msf','msm','mso']
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in degrees:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
degree_dict['MSC'].update(apl)
#------------------------------------------------------------------------------------------------------
degree_dict['MSC'].update(degree_dict['MS'])
del degree_dict['MS']
#------------------------------------------------------------------------------------------------------
anyinlist = ['law','llb']
allinlist = []
notinlist = ['llbr','mast','pg','post']
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in degrees:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
degree_dict['Bachelor of Laws'].update(apl)
#------------------------------------------------------------------------------------------------------
anyinlist = ['law','llm','mlb']
allinlist = []
notinlist = ['llbr','bach','dipl','com','inter','secon','law/tax']
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in degrees:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
degree_dict['Master of Laws'].update(apl)
#------------------------------------------------------------------------------------------------------


In [143]:
anyinlist = ['law','llm','mlb']
allinlist = []
notinlist = ['llbr','bach','dipl','com','inter','secon','law/tax']
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in degrees:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
# degree_dict['Master of Laws'].update(apl)

{'aca acs aicwa finance governance corporate law',
 'acs indian corporate laws and secretarial practice',
 'ba hons law  economics',
 'ba llb labour laws',
 'ballb law',
 'bank and financial management banking corporate finance and securities law',
 'bba llb business and law',
 'bgl law',
 'ca accounts tax costing and law',
 'caiib banking corporate finance and securities law',
 'chartered accountant accounting audit tax law',
 'chartered accountant accounts finance audit business laws',
 'chartered accountant accounts finance corporate law direct  indirect taxes costing',
 'chartered accountant acocunts audit taxation law',
 'chartered accountant finance banking audit risk securities laws',
 'chartered accountant financeaccountslaw',
 'chartered accountant financial accounting  reporting gaap financial management corporate laws direct  indirect tax',
 'chartered accountant financial managementcorporate lawsauditingtaxationbusiness managementcost control',
 'cpa accounting tax regulati

In [144]:
degree_dict.keys()

dict_keys(['MBA', 'BE', 'BCOM', 'BBA', 'CA', 'PGDBA', 'MTECH', 'MCOM', 'BA', 'MA', 'MCA', 'BCA', 'BSC', 'MSC', 'Bachelor of Laws', 'Master of Laws'])

In [145]:
converted = pd.DataFrame.from_dict(degree_dict, orient='index').T
converted.to_csv('converted.csv',index = False)

In [146]:
df[df['name']=='siva muthukaruppan'][[col for col in df.columns if 'digree' in col ]]

,digree_0,digree_0_duration,digree_1,digree_1_duration,digree_2,digree_2_duration,digree_3,digree_3_duration,digree_4,digree_4_duration,...,digree_16,digree_16_duration,digree_17,digree_17_duration,digree_18,digree_18_duration,digree_19,digree_19_duration,digree_20,digree_20_duration
25,bachelor of engineering be mechanical enginee...,Less than 1 year,bachelor of engineering be mechanical enginee...,Less than 1 year,nan,Less than 1 year,nan,Less than 1 year,nan,Less than 1 year,...,nan,Less than 1 year,nan,Less than 1 year,nan,Less than 1 year,nan,Less than 1 year,nan,Less than 1 year


In [147]:
for i in degree_dict['BE']:
    print(i)

bachelor of technology  btech aerospace engineering
be hons chemical
bengg electronics
be electronic and communication engineering
xii  west bengal council of higher secondary education science
bachelor of technology  btech engineering
bachelor of technology btech ocean engineering and naval architecture
bachelor of engineering  be electronics and instrumentation engineering
bachelor of engineering information science
be electronics  instrumentation
bachelor of technology btech electronics and instrumentation engineering
be mech engineering
bachelor of business economics bbe international finance  international business
bachelor of engineering be electronics and communication
bachelor of engineering electrical and electronics
b. tech.
be electrical and electronics
be computer science
be electrical engineering
bachelor of engineering be mumbai university information technology
be  civil engineering
be electronics
bachelor of technology  btech electronics and communications engineering
b

In [148]:
# for k,v in degree_dict.items():
#   degree_dict[k] = set([x.lower() for x in v])# make values to lowercase.

In [149]:
def map_degree(value):
    for k, v in degree_dict.items():
        if value in v:
            return k
    return 'others'
for col in degree_cols:
  df[f'mapped_{col}'] = df[col].apply(map_degree)

In [150]:
# dic = {}
# def count_degree(value):
#     for k, v in degree_dict.items():
        
#         if v.lower() == value:
#             dic[k] = dic.get(k, 0) + 1
# df[degree_cols].applymap(count_degree)
# dic
mapdegree = [i for i in df.columns if "digree" in i and 'map' in i]
value_counts = df[mapdegree].stack().value_counts().to_dict()
converted = pd.DataFrame.from_dict(value_counts, orient='index', columns=['Count'])
converted.index.name = "degree"
converted.to_excel('degree.xlsx')

In [151]:
df.shape

(7657, 916)

In [152]:
# c = 0
# for degree in cl:
#   c += 1
#   print(f"{degree} - {degree_counts[degree]}")

In [153]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7657 entries, 0 to 7656
Columns: 916 entries, name to mapped_digree_20
dtypes: float64(75), object(841)
memory usage: 53.5+ MB


In [154]:
columns_with_sets = [col for col in df.columns if df[col].apply(type).eq(set).any()]

for i in columns_with_sets:
  rows_with_sets = df[df[i].apply(type).eq(set)][columns_with_sets]
  print(rows_with_sets)

In [155]:
# value_counts = {}

# for col in df[degree_cols].columns:
#     counts = df[f'mapped_{col}'].value_counts()
#     for value, count in counts.items():
#         if value in value_counts:
#             value_counts[value] += count
#         else:
#             value_counts[value] = count
# value_counts

In [156]:
# df.drop_duplicates(inplace=True)
# df.reset_index(drop=True, inplace=True)
# df.to_excel('Linkedin_cleaned.xlsx',index = False)

In [157]:
df.shape

(7657, 916)

# degree cols done.

In [158]:
df.to_csv('checkp1.csv',index = False)

# Checkpoint

In [159]:
df=pd.read_csv('checkp1.csv')

C:\Users\PM\AppData\Local\Temp\ipykernel_16460\2479541234.py:1: DtypeWarning: Columns (6,7,9,10,11,13,14,15,17,18,19,21,22,23,25,26,27,29,30,31,33,34,35,37,38,39,41,42,43,45,50,51,53,54,55,57,58,59,61,62,63,65,66,67,69,70,71,73,74,75,77,78,79,81,82,83,85,86,87,89,94,95,97,98,99,101,102,103,105,106,107,109,110,111,113,114,115,117,118,119,121,122,123,125,126,127,129,130,131,133,154,155,157,158,159,161,162,163,165,166,167,169,170,171,173,174,175,177,178,179,181,182,183,185,186,187,189,190,191,193,202,203,205,206,207,209,210,211,213,214,215,217,218,219,221,222,223,226,227,230,231,234,235,237,238,239,241,242,243,245,246,247,249,250,251,253,254,255,262,263,265,266,267,269,270,271,273,274,275,277,278,279,281,282,283,285,286,287,289,290,291,293,294,295,297,298,299,301,302,303,305,306,307,310,311,314,315,317,318,319,321,322,323,325,326,327,329,330,331,333,334,335,337,342,343,345,346,347,349,350,351,353,354,355,357,358,359,361,362,363,365,366,367,369,370,371,373,374,375,377,378,379,381,390,391,3

In [160]:
df.columns = df.columns.str.replace(r'\W+', '_', regex=True).str.replace(r'_+', '_', regex=True).str.strip('_').str.lower()

In [161]:
###-------------------------------------------------------deletion procedure-------------------------------------------------------###

df.drop(columns=['employee_count','current_org_industry'], errors='ignore', inplace=True)
df.drop(columns=[col for col in df.columns if 'company' in col and 'emp' in col and 'map' not in col],errors='ignore',inplace=True)
df.drop(columns=[col for col in df.columns if 'company' in col and 'indu' in col and 'map' not in col],errors='ignore',inplace=True)
df.drop(columns=[col for col in df.columns if 'digree' in col and 'map' not in col and 'dura' not in col],errors='ignore',inplace=True)
df.drop(columns=[col for col in df.columns if 'insti' in col and 'map' not in col],errors='ignore',inplace=True)
df.drop(columns=[col for col in df.columns if 'title' in col and 'map' not in col],errors='ignore',inplace=True)
#------------------------------------------------------------------------------------------------------
subli = ['_sub_' + str(i) for i in range(1, 19)]
delcollis = [col for col in df.columns if any(sub in col for sub in subli) and 'title' in col] # works on mapped titles as titles are removed already.
df.drop(columns=delcollis, errors='ignore', inplace=True)
#------------------------------------------------------------------------------------------------------
for k,v in df[[col for col in df.columns if 'title' in col]].iterrows(): # copy titles from sub to orig.
    cols = [col for col in v.index.to_list() if 'sub' in col]
    for i in cols:
        orj = i.replace('_sub_0','')
        if (v[i] != 'others') and (type(v[i]) != float):
            v[orj] = v[i]
            df.at[k, orj] = v[i]
df.drop(columns=[col for col in df.columns if 'title' in col and 'sub' in col],errors='ignore',inplace=True) # delete after copy.
#------------------------------------------------------------------------------------------------------
delcollis = [col for col in df.columns if any(sub in col for sub in subli) and 'org' in col] # --------------- delete org cols with sub_1 till 19.
df.drop(columns=delcollis, errors='ignore', inplace=True)
#------------------------------------------------------------------------------------------------------
for k,v in df[[col for col in df.columns if 'org' in col]].iterrows(): # copy titles from sub to orig.
    cols = [col for col in v.index.to_list() if 'sub' in col]
    for i in cols:
        orj = i.replace('_sub_0','')
        if type(v[i]) != float:
            v[orj] = v[i]
            df.at[k, orj] = v[i]
df.drop(columns=[col for col in df.columns if 'org' in col and 'sub' in col],errors='ignore',inplace=True)
#------------------------------------------------------------------------------------------------------
delcollis = [col for col in df.columns if any(sub in col for sub in subli) and 'job' in col and 'loc' in col]
df.drop(columns=delcollis, errors='ignore', inplace=True)
#------------------------------------------------------------------------------------------------------
for k,v in df[[col for col in df.columns if 'job' in col and 'loc' in col]].iterrows(): # copy titles from sub to orig.
    cols = [col for col in v.index.to_list() if 'sub' in col]
    for i in cols:
        orj = i.replace('_sub_0','')
        if type(v[i]) != float:
            v[orj] = v[i]
            df.at[k, orj] = v[i]
df.drop(columns=df[[col for col in df.columns if 'job' in col and 'loc' in col and 'sub' in col]],errors='ignore',inplace=True)
#------------------------------------------------------------------------------------------------------
for k,v in df[[col for col in df.columns if 'job' in col and 'dura' in col]].iterrows():
    sub_cols = [col for col in v.index.to_list() if 'sub' in col]
    for i in range(21):
        if not v[[col for col in sub_cols if 'job_'+str(i)+'_' in col and 'sub_0' in col]].isna().all():
            if np.nansum(v[[col for col in sub_cols if 'job_'+str(i)+'_' in col]].to_numpy())//12 > 15:
                v['job_'+str(i)+'_duration'] = v[[col for col in sub_cols if 'job_'+str(i)+'_' in col and 'sub_0' in col]]//12
            else:
                v['job_'+str(i)+'_duration'] = np.nansum(v[[col for col in sub_cols if 'job_'+str(i)+'_' in col]].to_numpy())//12
df.drop(columns=df[[col for col in df.columns if 'job' in col and 'dura' in col and 'sub' in col]],errors='ignore',inplace=True)
#------------------------------------------------------------------------------------------------------


In [162]:
###-------------------------------------------------------Nan Reduction-------------------------------------------------------###

for i in df[[col for col in df.columns if 'org' in col or 'title' in col or 'job' in col or 'comp' in col]]: 
    n = int(re.sub(r'[^0-9]', '', i))
    if n > 7:
        del df[i]
#------------------------------------------------------------------------------------------------------
for i in df[[col for col in df.columns if 'digr' in col or 'inst' in col]]: 
    n = int(re.sub(r'[^0-9]', '', i))
    if n > 3:
        del df[i]
#------------------------------------------------------------------------------------------------------
for i in df[[col for col in df.columns if 'skill' in col]]: 
    n = int(re.sub(r'[^0-9]', '', i))
    if n > 19:
        del df[i]


In [163]:
for k,v in df[[col for col in df.columns if 'org' in col]].iterrows():
    df.at[k,'switches'] = v.notna().sum()

In [164]:
{re.sub(r'[^A-Za-z_]', '', col) for col in df.columns}

{'digree__duration',
 'job__duration',
 'job__location',
 'location',
 'mapped_company__emp_count',
 'mapped_company__industry',
 'mapped_digree_',
 'mapped_institute_',
 'mapped_title_',
 'name',
 'org_',
 'skill_',
 'switches',
 'tier_mapping_institute_',
 'total_exp'}

In [165]:
# c = 0
# for k,v in df[job_duration_cols].iterrows():
#     if np.nansum(v.to_numpy())//12 > 50:
#         c+=1
#         print(c,k,df.loc[k,'name'])

In [166]:
job_duration_cols = [col for col in df.columns if 'job' in col and 'duration' in col]

In [167]:
# def duracata(tot):
#     if pd.isna(tot):
#         return 'NAN'
#     tot = tot/12
#     if tot < 1:
#         return "Less than 1 year"
#     elif tot <= 6:
#         return f"{int(tot)} years"
#     else:
#         return "Greater than 6 years"

# for cols in job_duration_cols:
#     df[cols]=df[cols].apply(duracata)

In [168]:
df = df.replace('NAN',np.nan)
df = df.replace('others',np.nan)

In [169]:
df['mapped_title_0'].unique()

array(['founder', 'founder and ceo', 'ceo', nan, 'cfo', 'president',
       'project manager', 'consultant', 'CTO', 'product manager',
       'software developer', 'account manager', 'head-hr',
       'business analyst', 'CMO'], dtype=object)

In [170]:
indset = set()
for col in df[[i for i in df.columns if 'indu' in i]].columns:
    indset.update(set(df[col].unique().tolist()))
indset

{'construction',
 'consumer',
 'education',
 'financial services',
 'fmcg',
 'government',
 'healthcare',
 'hospitality',
 'manufacturing',
 'media',
 nan,
 'planning and design',
 'professional services',
 'publishing',
 'real estate',
 'resources',
 'software',
 'sports',
 'telecommunications',
 'transport, supply chain and logistics'}

In [171]:
industry_mapping = {
    'construction': 'Construction',
    'consumer': 'Consumer',
    'education': 'Education',
    'financial services': 'Financial Services',
    'fmcg': 'FMCG',
    'government': 'Government',
    'healthcare': 'Healthcare',
    'hospitality': 'Hospitality',
    'manufacturing': 'Manufacturing',
    'media': 'Media',
    'non-profit': 'Non-Profit',
    'planning and design': 'Planning and Design',
    'professional services': 'Professional Services',
    'publishing': 'Publishing',
    'real estate': 'Real Estate',
    'resources': 'Resources',
    'software': 'Software',
    'sports': 'Sports',
    'telecommunications': 'Telecommunications',
    'transport, supply chain and logistics': 'Transport, Supply Chain, and Logistics'
}

for col in df[[i for i in df.columns if 'indu' in i]].columns:
    df[col].replace(industry_mapping, inplace=True)

In [172]:
df[[col for col in df.columns if 'tier' in col]] = df[[col for col in df.columns if 'tier' in col]].astype(str)

In [173]:
titledict.keys()

dict_keys(['founder', 'ceo', 'cfo', 'product manager', 'project manager', 'head-hr', 'founder and ceo', 'business analyst', 'consultant', 'account manager', 'president', 'software developer', 'CTO', 'CMO'])

In [174]:
# for degree in degree_dict.keys():
#     digdf = df[df[[col for col in df.columns if 'digr' in col and 'dura' not in col]].apply(lambda col: col.str.contains(degree, na=False)).any(axis=1)]
#     for tier in ['1.0','2.0','3.0','4.0']:
#         tierdf = digdf[digdf[[col for col in df.columns if 'tier' in col]].apply(lambda col: col.str.contains(tier, na=False)).any(axis=1)]
#         for title in titledict.keys():
#             temp = tierdf[tierdf[[col for col in df.columns if 'title' in col]].apply(lambda col: col.str.contains(title, na=False)).any(axis=1)]
#             if not temp.empty and temp.shape[0] > 20:
#                 temp.to_excel(f'combination mappings/{degree}_{tier}_{title}.xlsx',index=False)
#             else:
#                 file_path = 'combination mappings/less_or_nodata.txt'
#                 try:
#                     with open(file_path, 'a') as file:
#                         file.write(f'{degree}_{tier}_{title}\n')
#                 except FileNotFoundError:
#                     # File doesn't exist, create and write
#                     with open(file_path, 'w') as file:
#                         file.write(f'{degree}_{tier}_{title}\n')

In [175]:
titset = set()
for col in df[[i for i in df.columns if 'title' in i]].columns:
    titset.update(set(df[col].unique().tolist()))
titset

{'CMO',
 'CTO',
 'account manager',
 'business analyst',
 'ceo',
 'cfo',
 'consultant',
 'founder',
 'founder and ceo',
 'head-hr',
 nan,
 'president',
 'product manager',
 'project manager',
 'software developer'}

In [176]:
mapper = {
    'account manager': 'Account Manager',
    'business analyst': 'Business Analyst',
    'ceo':'CEO',
    'founder':'Founder',
    'cfo': 'CFO',
    'consultant': 'Consultant',
    'founder and ceo': 'Founder and CEO',
    'head-hr': 'Head of HR',
    'president': 'President',
    'product manager': 'Product Manager',
    'project manager': 'Project Manager',
    'software developer': 'Software Developer'
}
for col in df[[i for i in df.columns if 'title' in i]].columns:
    df[col].replace(mapper, inplace=True)

In [177]:
df[[i for i in df.columns if 'digree' in i and 'dura' not in i]].loc[8]

mapped_digree_0    BCOM
mapped_digree_1    BCOM
mapped_digree_2     NaN
mapped_digree_3     NaN
Name: 8, dtype: object

In [178]:
for inde,row in df[[i for i in df.columns if 'digree' in i and 'dura' not in i]].iterrows():
    l = row.tolist()
    for i in l:
        if not isinstance(i,float):    
            if l.count(i) > 1:
                # print(inde,l)
                mapind = 'mapped_digree_'+str(len(l) - l[::-1].index(i) - 1) 
                df.at[inde,mapind] = np.nan 
                break
for inde,row in df[[i for i in df.columns if 'digree' in i and 'dura' not in i]].iterrows():
    l = row.tolist()
    for i in l:
        if not isinstance(i,float):    
            if l.count(i) > 1:
                # print(inde,l)
                mapind = 'mapped_digree_'+str(len(l) - l[::-1].index(i) - 1) 
                df.at[inde,mapind] = np.nan 
                break

In [179]:
titlesorder = set()
for col in df[[i for i in df.columns if 'title' in i]]:
    titlesorder.update(set(df[col].unique().tolist()))
titlesorder = {digree for digree in titlesorder if pd.notna(digree)}
titlesorder

{'Account Manager',
 'Business Analyst',
 'CEO',
 'CFO',
 'CMO',
 'CTO',
 'Consultant',
 'Founder',
 'Founder and CEO',
 'Head of HR',
 'President',
 'Product Manager',
 'Project Manager',
 'Software Developer'}

In [180]:
title_cols = [col for col in df.columns if 'title' in col]
degree_cols = [col for col in df.columns if 'digree' in col and 'dura' not in col]
tier_cols = [col for col in df.columns if 'tier' in col]

In [181]:
digreeorder = set()
for col in df[degree_cols]:
    digreeorder.update(set(df[col].unique().tolist()))
digreeorder = {digree for digree in digreeorder if pd.notna(digree)}
digreeorder

{'BA',
 'BBA',
 'BCA',
 'BCOM',
 'BE',
 'BSC',
 'Bachelor of Laws',
 'CA',
 'MA',
 'MBA',
 'MCA',
 'MCOM',
 'MSC',
 'MTECH',
 'Master of Laws',
 'PGDBA'}

In [182]:
title_value_points = {
    'Account Manager': 2,
    'Business Analyst': 1,
    'CEO': 3,
    'CFO': 3,
    'CMO': 3,
    'CTO': 3,
    'Consultant': 1,
    'Founder': 3,
    'Founder and CEO': 3,
    'Head of HR': 2,
    'President': 3,
    'Product Manager': 2,
    'Project Manager': 2,
    'Software Developer': 1
}

In [183]:
df['rare_or_normal'] = 'normal'
count = 0
for k,v in df[[i for i in df.columns if 'title' in i]].iterrows():
    v = v.to_list()
    c = [title_value_points[i] for i in v if i in title_value_points]
    d = sorted(c,reverse=True)
    if c != d:
        df.at[k,'rare_or_normal'] = 'rare'
        # print(count+1,df.loc[k].to_list())
        count+=1

In [197]:
df = df[(df['total_exp'] > 7) & (df['switches'] > 2)]
df.reset_index(drop=True,inplace=True)

In [201]:
datacountdict = []
for title in titlesorder:
    subdic = {}
    subdic['title'] = title
    titlefildf = df[(df[title_cols].isin([title])).any(axis=1)]
    for degree in digreeorder:
        degreedf = titlefildf[titlefildf[degree_cols].isin([degree]).any(axis=1)]
        for tier in ['1','2','3','4']:
            subdic[degree+' '+tier] = degreedf[degreedf[tier_cols].isin([tier]).any(axis=1)].shape[0]
    datacountdict.append(subdic)

In [202]:
datacountdf = pd.DataFrame(datacountdict)

In [203]:
datacountdf.columns = ['\n'.join([' '.join(col.split(' ')[:-1]),''.join(col.split(' ')[-1:])]) for col in datacountdf.columns]

In [204]:
datacountdf.to_excel('data_counts.xlsx',index = False)

In [205]:
df.to_excel('Linkedin_cleaned_latest_mapped_cols.xlsx',index=False)

In [206]:
import mysql.connector
from sqlalchemy import create_engine
# Define your database connection parameters
host = "54.79.186.42"
user = "root"
password = "secondstorey"
database = "secondstorey"

# Establish a connection to the database
connection = mysql.connector.connect(
    host=host,
    user=user,
    password=password,
    database=database
)

# Check if the connection was successful
if connection.is_connected():
    print("Connected to MySQL database")

cursor = connection.cursor()
cursor.execute("SHOW DATABASES")

databases = cursor.fetchall()
for db in databases:
    print(db[0])

Connected to MySQL database
azkroflyz
gmat
information_schema
mysql
performance_schema
salary_app
secondstorey
sys


In [207]:
engine = create_engine(f"mysql+mysqlconnector://{user}:{password}@{host}/{database}")
df.to_sql(name='Linkedin_data', con=engine, if_exists='replace', index=False)

3300